In [12]:
full_info = {}

In [1]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:

import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [3]:
import docx

def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

In [4]:
document = getText('C:\\Users\\azhar\\Desktop\\Docssss\\OfferletterMd.docx')

In [5]:
import re
lines_ini = re.split('    |\n',document)
lines_ini = [el.strip(',') for el in lines_ini] # from each line removing comma (,)

In [6]:
lines = [nltk.word_tokenize(el) for el in lines_ini] # tokenizing each word from each line,
lines = [nltk.pos_tag(el) for el in lines] # applying pos tag one for each word line by line.
lines = [x for x in lines if x != []] # finally making a list of list of tokenized word.
lines = lines[1:-3] 

In [7]:
#lines

In [8]:
grammer = r'NAME: {^<NN.*><NN.*>?<NN.*>?$}'
chunkParser = nltk.RegexpParser(grammer)

In [9]:
nameHits = []
for tagged_tokens in lines:
    chunked_tokens = chunkParser.parse(tagged_tokens)
    #print(chunked_tokens)
    for subtree in chunked_tokens.subtrees():
        if subtree.label() == 'NAME':
            nameHits.append(' '.join([i[0] for i in subtree.leaves()]))

In [13]:
full_info['Candidate Name'] = nameHits[0]
if full_info['Candidate Name'].find('Date')!=-1:
    full_info['Candidate Name'] = nameHits[1]
    
full_info['offerer Name'] = nameHits[-1]
if nameHits[-1] == 'Hiring Manager':
    full_info['offerer Name'] = nameHits[-2]

In [14]:
for l in lines_ini:
    if l.lower().find('dear')!= -1 and len(l) > 8:
        full_info['Candidate Name'] = l[(l.lower().find('dear') + 5):]
        break

In [15]:
import spacy
nlp = spacy.load('en')

def extract_join_dt_sal_org(lines):
    dates = []
    money = []
    org = []
    loc = []
    time= []
    #docu=[]
    for line in lines:
        doc=nlp(str(line))
        for ent in doc.ents:
            if ent.label_ == 'ORG':
                    org.append(str(ent.text))
            if ent.label_ == 'MONEY':
                    money.append(str(ent.text))
            if ent.label_ == 'DATE':
                    match = re.search(r'(^\d{1,2}\D.*\d{4}$)',ent.text.strip())
                    if match != None:
                        dates.append(str(ent.text))
            if ent.label_ == 'GPE':
                    loc.append(ent.text)
            if ent.label_ == 'TIME':
                time.append(ent.text)
            #if ent.label_ == 'LAW':
                #docu.append(ent.text)    
    
    if not dates:
        dates = ['not found']
    if not money:
        money = ['Confidential']
    if not org:
        org = ['not found']
    if not loc:
        loc=['not found']
    if not time:
        time=['Not Found']
    #if not docu:
        #docu=['Not Mentioned']    
    return dates[-1],money[0],org[0],loc[0],time[0]#,docu[0]

In [16]:
join_date,salary,org,loc,time=extract_join_dt_sal_org(lines_ini)
full_info['Joining date'] = join_date
full_info['Salary'] = salary
full_info['Organisation Name'] = org
full_info['Location']= loc
full_info['Time per day/weekly']=time


In [17]:
import find_job_titles
from find_job_titles import FinderAcora
b=FinderAcora()
gen=FinderAcora.find_raw(b,document)
designation=[]
for l in gen:
    designation.append(l)
    break

ModuleNotFoundError: No module named 'find_job_titles'

In [16]:

if not designation:
    full_info['Designation'] = 'Not found'
else:
    des= designation[0]
    d = document[des.start-15:des.end+15]
    if document.find('Trainee')!=-1:
        ind = document.find('Trainee')
        d = document[ind-20:ind+27]
    if d == '':
        d = document[des.start:des.end+15]
        
    d = nltk.word_tokenize(str(d))
    d = nltk.pos_tag(d)
    grammer = r'DESIGNATION: {<NNP>*}'
    chunkParser = nltk.RegexpParser(grammer)
    desHits = []
    chunked_tokens = chunkParser.parse(d)
    for subtree in chunked_tokens.subtrees():
        if subtree.label() == 'DESIGNATION':
            word=' '.join([i[0] for i in subtree.leaves()])
            if document.find('Trainee')!=-1:
                desHits.append(word)
            elif des.match in word:
                desHits.append(word)
    if not desHits:
        full_info['Designation'] = 'Not found'
    else:
        full_info['Designation'] = desHits[0]

In [17]:
if ((full_info['Designation']).strip() == (full_info['Organisation Name']).strip()):
    full_info['Organisation Name'] = 'Not Found'

In [18]:
full_info

{'Candidate Name': ' Azhar',
 'offerer Name': 'David',
 'Joining date': '20/09/2018',
 'Salary': '$3000 dollar',
 'Organisation Name': 'CodeLogix',
 'Location': 'Bangalore city',
 'Time per day/weekly': ' 9:00 AM to 6:00 PM',
 'Designation': 'Software Engineer Trainee'}